# Loading sample data into Delta Lake on ADLS 2



# Download the sample data

In [ ]:
from pyspark import SparkFiles
file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
spark.sparkContext.addFile(file_url)

heart = spark.read.csv("file://"+SparkFiles.get("heart.csv"), header=True, inferSchema= True)
display(heart)

# Write data into Delta Lake
Pre-requsities: </br>
1. Create a ADLS Gen 2 storage account </br>
2. Create a Service Principle - https://docs.microsoft.com/en-us/azure/databricks/data/data-sources/azure/azure-storage#--access-azure-data-lake-storage-gen2-or-blob-storage-using-oauth-20-with-an-azure-service-principal

In [ ]:
write_format = 'delta'
write_mode = 'overwrite'
save_path = "abfss://<blob container>@<storage-account>.dfs.core.windows.net/"
peopleTablePath = save_path + 'delta-tables/heart'

spark.conf.set("fs.azure.account.auth.type.<storage-account>.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.<storage-account>.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.<storage-account>.dfs.core.windows.net", "<client-id>")
spark.conf.set("fs.azure.account.oauth2.client.secret.<storage-account>.dfs.core.windows.net", "<secret>")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.<storage-account>.dfs.core.windows.net", "https://login.microsoftonline.com/<tenant-id>/oauth2/token")

In [ ]:

# Write the data to its target.
heart.write \
  .format(write_format) \
  .mode(write_mode) \
  .saveAsTable("heart_data", path=peopleTablePath)
